# Get the wiki data

In [1]:
import requests
html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'lxml')

In [3]:
table = soup.table
tds = table.find_all('td')

In [4]:
# map the td text to a dataframe
df = []
obj = {}
count = 0
for d in tds:
    if count == 0:
        obj['Postcode'] = d.text
        count = count + 1
    elif count == 1:
            obj['Borough'] = d.text
            count = count + 1
    elif count == 2:
            obj['Neighbourhood'] = d.text.replace('\n', '')
            count = 0
            df.append(obj)
            obj = {}

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(df)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


# Data Preprocessing

In [7]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.Borough.value_counts()

Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [8]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df[df.Neighbourhood == 'Not assigned'] = df['Borough']

In [9]:
# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
dfGroup = df.groupby('Postcode')
df_singlePostcode = pd.DataFrame()
for g in dfGroup:
    str = ','.join(g[1].Neighbourhood.unique())
    g[1].loc[:,'Neighbourhood'] = str
    df_singlePostcode = pd.concat([df_singlePostcode, g[1]])

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [10]:
df_singlePostcode = pd.DataFrame(df_singlePostcode.drop_duplicates())

In [11]:
df_singlePostcode.head()

,Borough,Neighbourhood,Postcode
11,Scarborough,"Rouge,Malvern",M1B
27,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C
42,Scarborough,"Guildwood,Morningside,West Hill",M1E
53,Scarborough,Woburn,M1G
62,Scarborough,Cedarbrae,M1H


In [12]:
df_singlePostcode.shape

(103, 3)

# Get the geo data

In [13]:
# read the csv file of geo data
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [14]:
# merge two dataframe
df_result = pd.merge(df_geo, df_singlePostcode, left_on='Postal Code', right_on='Postcode', how='right')

In [15]:
df_result.drop(['Postal Code'], inplace=True, axis=1)
df_result.head()

,Latitude,Longitude,Borough,Neighbourhood,Postcode
0,43.806686,-79.194353,Scarborough,"Rouge,Malvern",M1B
1,43.784535,-79.160497,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C
2,43.763573,-79.188711,Scarborough,"Guildwood,Morningside,West Hill",M1E
3,43.770992,-79.216917,Scarborough,Woburn,M1G
4,43.773136,-79.239476,Scarborough,Cedarbrae,M1H


# Map

In [16]:
import folium

In [17]:
df_result.dropna(axis=0, how='any', inplace=True)
latitude = df_result.Latitude.mean()
longitude = df_result.Longitude.mean()

In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(df_result.Latitude, df_result.Longitude, df_result.Postcode):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map